In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas
import math
import os
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import GRU
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.utils import plot_model

# fix random seed for reproducibility
np.random.seed(7)

/home/vlad/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
DATA_TRAIN_PATH = 'blood-pressure-estimation-data/data_train'
DATA_TRAIN_ANS_PATH = 'blood-pressure-estimation-data/data_train.ans'
DATA_TEST1_PATH = 'blood-pressure-estimation-data/data_test1_blank'
DATA_TEST2_PATH = 'blood-pressure-estimation-data/data_test2_blank'

In [3]:
def load_answers(path):
    sbp_dbp = {}
    with open(path, 'r') as f:
        for line in f:
            vals = line[:-1].split(',')
            sbp_dbp[vals[0]] = (float(vals[1]), float(vals[2]))
    return sbp_dbp

sbp_dbp = load_answers(DATA_TRAIN_ANS_PATH)

def read_ppg_ecg(filename):
    ppg = []
    ecg = []
    with open(filename, 'r') as f:
        for id, line in enumerate(f):
            if id == 0:
                continue
            vals = line[:-1].split(',')
            ppg.append(float(vals[0]))
            ecg.append(float(vals[1]))
    return ppg[2000:], ecg[2000:]

In [4]:
def prepareTrain(batch_size, timesteps, is_sbp = True):
    normX = 20000000
    normY = 200
    
    while True:
        X = None
        Y = None

        files = list(sbp_dbp.keys())
        for file in files:
            ppg, ecg = read_ppg_ecg(DATA_TRAIN_PATH + '/' + file)
            ppg = ppg[:timesteps]
            ecg = ecg[:timesteps]
            line = np.array([[list(x) for x in zip(ppg, ecg)]])
            line /= normX
            X = line if X is None else np.vstack([X, line])
            
            lineY = np.array([sbp_dbp[file][0] / normY])
            Y = lineY if Y is None else np.vstack([Y, lineY])
            if X.shape[0] == batch_size:
                #print(Y)
                yield X, Y
                X = None
                Y = None

        if X.shape[0] > 0:
            yield X, Y


def preparePredict(files, batch_size, timesteps):
    normX = 20000000
    
    X = None
    for file in files:
        ppg, ecg = read_ppg_ecg(DATA_TRAIN_PATH + '/' + file)
        ppg = ppg[:timesteps]
        ecg = ecg[:timesteps]
        line = np.array([[list(x) for x in zip(ppg, ecg)]])
        line /= normX
        X = line if X is None else np.vstack([X, line])
        if X.shape[0] == batch_size:
            #print(X)
            yield X
            X = None

    if X.shape[0] > 0:
        yield X

In [74]:
batch_size = 200
timesteps = 5000
train_size = len(sbp_dbp)
print(train_size)
for i, v in enumerate(prepareTrain(batch_size, timesteps)):
    for j in range(v[1].shape[0]):
        print(v[1][j][0] * 2000, v[1][j][1] * 2000)
        break

894
1260.0 790.0
1370.0 620.0


KeyboardInterrupt: 

In [6]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 7759000777594865978
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 2654339072
locality {
  bus_id: 1
  links {
  }
}
incarnation: 3076674917756822408
physical_device_desc: "device: 0, name: GeForce GTX 1060 3GB, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


In [2]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

['/job:localhost/replica:0/task:0/device:GPU:0']

In [5]:
batch_size = 100
timesteps = 1000
train_size = len(sbp_dbp)

model = Sequential()
model.add(LSTM(500, input_shape=(timesteps, 2), dropout=0.2, recurrent_dropout=0.2, return_sequences=True))
model.add(LSTM(96, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit_generator(prepareTrain(batch_size, timesteps), epochs=1, steps_per_epoch= (train_size + batch_size - 1) // batch_size)

Epoch 1/1
9/9 [==============================] - 28s 3s/step - loss: 0.1143


In [6]:
test_list = os.listdir(DATA_TRAIN_PATH)
predictions = model.predict_generator(preparePredict(test_list, batch_size, timesteps), steps = (len(test_list) + batch_size - 1) // batch_size)

In [7]:
#predictions.resize(len(final_list))
for v in predictions:
    print(v * 200)
print(len(predictions))
print(len(test_list))

[72.17256]
[70.78644]
[70.45933]
[71.41137]
[71.92926]
[71.51066]
[70.45826]
[71.996155]
[71.92425]
[71.59408]
[71.31174]
[71.54567]
[71.26435]
[71.04506]
[70.423805]
[71.67932]
[71.206535]
[71.95038]
[70.52438]
[70.71374]
[71.7481]
[71.48966]
[71.90833]
[72.061714]
[71.99927]
[72.05885]
[70.69977]
[71.464]
[71.56974]
[71.94023]
[70.8819]
[71.95745]
[71.811386]
[72.00939]
[71.71038]
[70.48652]
[71.61366]
[71.38605]
[71.81119]
[72.04962]
[71.75663]
[70.82857]
[71.9412]
[70.79471]
[71.071014]
[71.00395]
[71.25123]
[70.655785]
[71.9911]
[71.4157]
[70.39927]
[71.56834]
[71.17027]
[71.35371]
[71.55555]
[70.99168]
[71.496086]
[71.90482]
[70.69]
[71.95703]
[72.06167]
[70.68608]
[71.98711]
[71.18393]
[70.62512]
[71.96737]
[71.04438]
[70.66638]
[70.6699]
[71.56997]
[71.70134]
[70.64764]
[71.392105]
[71.246735]
[70.5191]
[71.27562]
[70.514175]
[71.2691]
[70.71356]
[70.72175]
[71.36071]
[70.806656]
[72.03574]
[71.90305]
[70.659225]
[71.935974]
[70.71817]
[70.59769]
[70.985886]
[71.93939]
[71.5350

In [55]:
test_list = os.listdir(DATA_TEST1_PATH)
predictions = model.predict_generator(preparePredict(test_list, batch_size, timesteps), steps = (len(test_list) + batch_size - 1) // batch_size)

In [56]:
#predictions.resize(len(final_list))
print(predictions * 2000)
print(len(predictions))
print(len(test_list))

[[181.26682 181.47841]
 [182.12346 182.49228]
 [180.84091 180.9778 ]
 [178.55505 178.28502]
 [178.52493 178.24464]
 [178.66295 178.41747]
 [181.9684  182.3069 ]
 [181.63315 181.91194]
 [180.82474 180.9571 ]
 [181.8352  182.15123]
 [180.82861 180.95949]
 [182.10529 182.46957]
 [181.87944 182.20421]
 [181.7169  182.01091]
 [179.17915 179.03189]
 [179.84338 179.79825]
 [179.30684 179.1653 ]
 [178.6082  178.34297]
 [178.58247 178.31276]
 [178.90942 178.6964 ]
 [181.86351 182.1855 ]
 [179.00839 178.8276 ]
 [181.70618 181.99913]
 [181.51587 181.77298]
 [178.54341 178.27579]
 [177.91339 177.52861]
 [179.35721 179.22612]
 [179.25063 179.1119 ]
 [178.85944 178.64452]
 [178.9698  178.77153]
 [177.09785 176.5621 ]
 [181.028   181.19788]
 [180.9026  181.05003]
 [181.02954 181.20828]
 [180.48093 180.55034]
 [178.56786 178.29642]
 [178.88055 178.69008]
 [180.94917 181.10385]
 [180.77266 180.89354]
 [181.77956 182.08455]
 [180.66574 180.76485]
 [178.95238 178.75612]
 [182.00883 182.35594]
 [181.67992